## Przygotowanie


Rozpoczniemy Od przygotowania środowiska - jeśli używasz notatnika w środowisku Colab - aktualizacja biblioteki "statsmodels" do wersji 0.12. 

In [ ]:
!pip install statsmodels --upgrade

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import statsmodels.api as sm
import itertools

keras = tf.keras

Następnie zdefiniujemy funkcje pomocnicze odpowiadające zarówno za tworzenie wykresów oraz za przetworzenie danych

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    plt.plot(time[start:end], series[start:end], format, label=label)
    ax = plt.gca()
    ax.xaxis.set_major_locator(plt.MaxNLocator(8))
    plt.xlabel("Time")
    plt.ylabel("Value")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)

def print_error(valid, predicted): 
    print('mean absoulte error:')
    print(keras.metrics.mean_absolute_error(valid, predicted).numpy())
    print('mean squared error')
    print(keras.metrics.mean_squared_error(valid, predicted).numpy())  

def sequential_window_dataset(series, window_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1))
    ds = ds.map(lambda window: (window[:-1], window[1:]))
    return ds.batch(1).prefetch(1)
  
def seq2seq_window_dataset(series, window_size, batch_size=32,
                           shuffle_buffer=1000):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

    


#Szereg czasowy
Szeregiem czasowym nazywamy uporządkowaną sekwencję, w której dane ułożone są sekwencyjnie, a kluczem określającym ich położenie jest czas.
W zależności od wartości określanych przez encjęw danym punkcie w czasie, dane możemy podzielić na:
- jednowymiarowe (jedna wartość w jednym punkcie)
- wielowymiarowe (wiele wartości w jednym punkcie)

#Występowanie
Szeregi czasowe występują dosłownie w każdej dziedzinie życia. Ich zastosowanie można łatwo dostrzec m.in w prognozach pogody, na giełdzie, w historycznych danych, pomiarach.



#Przykładowy zbiór danych jednowymiarowych.
W tym notatniku pokażemy metody predykcji szeregów czasowych na przykładzie szeregów jednowymiarowych. Przykładowy szereg przedstawiający ilość sprzedanych szamponów na przestrzeni 3 lat

In [ ]:
url = "https://raw.githubusercontent.com/Mervolt/TimeSeriesTutorial/master/shampoo.csv"

shampoo_dataset = pd.read_csv(url, error_bad_lines=False)
print(shampoo_dataset)

In [ ]:
time, values = shampoo_dataset["Month"], shampoo_dataset["Sales"]

plt.figure(figsize=(10, 6))
plot_series(time, values, label = False)
plt.show()

#Wspólne cechy szeregów

Wiele szeregów czasowych posiada takie właściwości jak:
- trend (np. monotoniczny wzrost lub spadek)

- sezonowość, którą można zaobserwować jako okres na wykresie (np. ilość turystów w zależności od miesiąca pokazywać będzie największe wartości w okresie letnim)
- szum, czyli zakłócenia, drobne błędy wartości występujące w zbiorze danych

Poniżej przedstawiono te 3 właściwości na wykresach. 

#Trend rosnący

In [ ]:
def trend(time, slope=0):
    return slope * time

time = np.arange(4 * 365 + 1)
baseline = 10
series = baseline + trend(time, 0.1)

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

#Sezonowość

In [ ]:
def seasonal_pattern(season_time):
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

amplitude = 40
series = seasonality(time, period=365, amplitude=amplitude)

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

#Szum

In [ ]:
def white_noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

noise_level = 5
noise = white_noise(time, noise_level, seed=42)

plt.figure(figsize=(10, 6))
plot_series(time, noise)
plt.show()

#Przewidywanie

Posiadając zbiór przedstawiający szereg czasowy pewnych wartości możemy dokonać próby oszacowania wartości jakie wystąpią w przyszłości.

Użyjemy zbioru przedstawiającego minimalne temperature w Melbourne w latach 1981-1990.

Zbiór podzielimy zgodnie z metodą "Fixed partitioning" na część do uczenia, walidacyjną i testującą. 

Wykorzystamy kilka metod - od prostych sztuczek jak naiwne przewidywanie, po bardziej złożone statystyczne metody jak ARIMA, następnie użyjemy sieci neuronowych do przewidywania danych. 

#Naiwne przewidywanie

Najprostszym możliwym sposobem jest uznanie poprzedniej wartości jako przewidywaną. Metoda wydaje się być prymitywna, ale osiągane przez nią rezultaty są warte zaobserwowania chociażby dla celów porównania z innymi metodami.



In [ ]:
url = "https://raw.githubusercontent.com/Mervolt/TimeSeriesTutorial/master/melbourne_min_temp.csv"

dataset = pd.read_csv(url)
print(dataset)

In [ ]:
split = 3000
time, values = dataset["Date"], dataset["Temp"]
x_train, y_train = time[:split], values[:split]
x_valid, y_valid = time[split:], values[split:] 

In [ ]:
naive_forecast = values[split - 1:-1]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, naive_forecast, label="Forecast")

Wykresy nachodzą na siebie w takim stopniu, że nie można ich od siebie odróżnić. Wydzielimy teraz dla celów demonstracyjnych podzbiór.

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, start=0, end=150, label="Values")
plot_series(x_valid, naive_forecast, start=1, end=151, label="Forecast")

Możemy zaobserwować, że przewidywania są po prostu 1 krok za rzeczywistymi wartościami.

W celu ewaluacji potrzebujemy metryk. Użyjemy w tym przypadku metryk średniokwadratowej oraz odległości w przestrzeni Euklidesowej.

In [ ]:
print_error(y_valid, naive_forecast)


#Ruchoma średnia
Podejściem, które teraz zostanie zaprezentowane to ruchoma średnia. Jest ono relatywnie proste i polega na wyciąganiu średniej z okresu o długości n. To "ruchome okno" o długości n przesuwamy po całym zbiorze danych.
Przykładowo dla okna o długości 3, wartość pola o indeksie 7 liczymy jako średnią z pól o indeksach 4, 5 i 6, a dla pola o indeksie 22 z pól o indeksach 19, 20 i 21.

Zalety:
- redukuje szum

Wady:
- nie uwzględnia sezonowości
- nie uwzględnia trendów

In [ ]:
def moving_average_forecast(values, window_size):
  forecast = []
  for time in range(len(values) - window_size):
    forecast.append(values[time:time + window_size].mean())
  return np.array(forecast)

In [ ]:
moving_avg = moving_average_forecast(values, 3)[split - 3:]

plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, moving_avg, label="Ruchoma średnia (3 dni)")

In [ ]:
print_error(y_valid, moving_avg)

Osiągneliśmy gorsze wyniki niż w przypadku naiwnego podejścia. Może długość aplikowanego okna była za mała?
Spróbujmy dla innej wielkości okna.

In [ ]:
moving_avg = moving_average_forecast(values, 10)[split - 10:]

plt.figure(figsize=(10, 6))
plot_series(x_valid, y_valid, label="Values")
plot_series(x_valid, moving_avg, label="Ruchoma średnia (10 dni)")

In [ ]:
print_error(y_valid, moving_avg)

Wciąż gorsze wyniki. Powodem jest tutaj potężna wada tego podejścia, a mianowicie brak uwzględnienia sezonowości, która w przypadku temperatur jest bardzo ważnym czynnikiem.

#Ulepszona metoda ruchomej średniej
Aby ulepszyć metodę ruchomej średniej należy zlikwidować jej wady - brak uwzględniania trendów oraz brak uwzględniania sezonowości.
W tym celu należy specjalnie zadaptować nasz zbiór danych. Zamiast korzystać po prostu z naszego zbioru, korzystać będziemy z różnic (wartość - wartość wcześniejsza o pewien czas t, np. 1 rok i różnica = czerwiec 1984 - czerwiec 1983 )

In [ ]:
diff_values = (values[365:].reset_index() - values[:-365].reset_index())['Temp']
diff_time = time[365:]
plt.figure(figsize=(10, 6))
plot_series(diff_time, diff_values, label="Values(t) – Values(t–365)")
plt.show()

Podzbiór walidacyjny

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(x_valid, diff_values[split - 365:], label="Values(t) – Values(t–365)")
plt.show()

In [ ]:
diff_moving_avg = moving_average_forecast(diff_values, 30)[split - 365 - 30:]

plt.figure(figsize=(10, 6))
plot_series(x_valid, diff_values[split - 365:], label="Values(t) – Values(t–365)")
plot_series(x_valid, diff_moving_avg, label="Moving Average of Diff")
plt.show()

W porządku. Obliczyliśmy ruchomą średnią, ale nie jest to nasz zbiór danych. W takim razie musimy go odzyskać.
Aby to zrobić należy dodać wartości z przeszłości.

In [ ]:
diff_moving_avg_plus_past = values[split - 365:-365] + diff_moving_avg

plt.figure(figsize=(10, 6))
plot_series(time, values, label="Values")
plot_series(x_valid, diff_moving_avg_plus_past, label="Forecasts")
plt.show()

In [ ]:
print_error(y_valid, diff_moving_avg_plus_past)

Otrzymaliśmy jeszcze gorsze wyniki. Spróbujmy zredukować szum w początkowych danych.

In [ ]:
diff_moving_avg_plus_smooth_past = moving_average_forecast(values[split - 370:-359], 11) + diff_moving_avg

plt.figure(figsize=(10, 6))
plot_series(time, values, label="Values")
plot_series(x_valid, diff_moving_avg_plus_smooth_past, label="Forecasts")
plt.show()

In [ ]:
print_error(y_valid, diff_moving_avg_plus_smooth_past)

Znacznie lepiej, jednakże wciąż otrzymaliśmy gorsze rezultaty niż w przypadku naiwnego przewidywania

# ARIMA

Innym podejściem, może być zastosowanie modelu ARIMA (Autoregressive Integrated Moving Average). 

Model ten składa się z trzech elementów: 
- proces autoregresyjny **AR**
- proces średniej ruchomej **MA**
- stopień integracji **I** 

Proces autoregresyjny oznacza, że każda wartość jest liniową kombinacją poprzednich wartości. Dla Modelu AR(p), dla p=1 proces można przedstawić następująco: 

$  y_t = y_{t-1} + \epsilon_t  $

Gdzie :

$ y_t $ - wartość szeregu w chwili t  

$ y_{t-1} $ - wartość szeregu w chwili t-1 

$ \epsilon_t $ - składnik losowy, zaburzenie w chwili t 

W procesie średniej ruchomej zakładamy, że wartość zalerzy od zaburzeń w chwili obecnej i wcześniejszych, np. model MA(p), dla p = 1 :

$  y_t = \epsilon_{t-1} + \epsilon_{t}  $

Gdzie :

$ y_t $ - wartość szeregu w chwili t  

$ \epsilon_{t-1} $ -  składnik losowy, zaburzenie w chwili t -1 

$ \epsilon_t $ - składnik losowy, zaburzenie w chwili t 

Integracja - oznacza sprowadzenie procesu do postaci stacjonarnej, np. wykres może wykazywać trend, a po integracji, będziemy posiadać wartości stacjonarne, tj. takie, które będą mieć stały poziom odchylenia oraz stały poziom średniej. 

Model ten uwzględnia 3 istotne parametry p d q, które wpływają na przewidywanie danych w zależności od sezonowości, trendu i szumu. Każdy z parametrów przyjmuje wartości dodatnie: 0,1,2... Natomiast wartość parametru równa 0 oznacza ignorowanie go, np. dla danych bez trendu można ustawić wartość parametru d na 0, przez co model ARIMA(a,0,b) będzie równoważny modelowi ARMA(a,b). Niektóre przykłady modelu ARIMA są odpowiednikami innych modeli, np. ARIMA(0,1,0) - to błądzenie losowe, ARIMA(0,0,0) - to szum biały. 



In [ ]:
p = q = range(0, 3)
d = [0,0,0,0]
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 365) for x in list(itertools.product(p, d, q))]

Najpierw zajmiemy się poszukiwaniem najlepszych parametrów p, d, q - takich aby wartość AIC była najniższa. Wykorzystamy do tego prostą metodę - sprawdzimy kombinacje p i q dla wartości od 0 do 2, oraz dla wartości d równej 0 - dane nie mają trendu, więc nasz model nie uwzględni parametru d (przyjmie on wartość 0). 

In [ ]:
min = 25000
min_params = None
for param in pdq:
      mod = sm.tsa.arima.ARIMA(values,order=param, enforce_stationarity=False, enforce_invertibility=False)
      results = mod.fit()
      if(results.aic < min):
        min = results.aic
        min_params = param
      print(f'ARIMA{param} - AIC:{results.aic}')

print(f'MIN: {min}')
print(f'Param: {min_params}')

In [ ]:
model = sm.tsa.arima.ARIMA(values,order=min_params)
results = model.fit()
print(results.summary().tables[1])

In [ ]:
predictions = results.get_prediction(split)

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time, values, label="Values")
plot_series(x_valid, predictions.predicted_mean, label="Forecast")

In [ ]:
print_error(y_valid, predictions.predicted_mean)

Jak widać, uzyskaliśmy najlepsze dotychczas wyniki

#Przewidywanie - Machine Learning
Użyjemy metody regresji liniowej, w oparciu o okno zawierające 30 dni. Nasza początkowa sieć składać się będzie z pojedynczej warstwy, bez użycia funkcji aktywacji. Użycie pędu do optymalizacji wyników z reguły zwiększa zbieżność metody. 

In [ ]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)
valid_set = window_dataset(y_valid, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(1, input_shape=[window_size])
])
optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
model.fit(train_set, epochs=150, validation_data=valid_set)

Sukces! Udało nam się przebić naiwne przewidywanie, jednak ARIMA dalej posiada lepsze wyniki.
Możemy spróbować poprawić nasze wyniki na kilka sposobów. Po pierwsze, spróbujemy sprawdzić jaka stała ucząca może być dla nas najlepsza, poprzez zwiększanie jej co pewną ilość epok. Dzięki temu zobaczymy kiedy model uczy się najszybciej. 

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(1, input_shape=[window_size])
])

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 30))
optimizer = keras.optimizers.SGD(lr=1e-6, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=120, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-6, 1e-2, 0, 20])

Jak widać na wykresie, spadek błędu był najlepszy dla wartości ok 1e-4. Dla większych wartości błąd zaczął stawać się o wiele większy. Dzięki temu możemy założyć jaka stała będzie najlepsza dla naszego modelu. 
Następną rzeczą jaką możemy poprawić to dołożenie do modelu specjalnej metody wczesnego zatrzymania, która sprawdza czy błąd zmienił się na lepsze przez ostatnie kilka epok, jeśli nie to uczenie modelu zostaje przerwane wcześniej. Dzięki temu możemy ustawić ilośc epok na znacznie większą, np. 500 a model będzie się uczył dopóki błąd będzie malał. 

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)
valid_set = window_dataset(y_valid, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(1, input_shape=[window_size])
])
optimizer = keras.optimizers.SGD(lr=1e-4, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
early_stopping = keras.callbacks.EarlyStopping(patience=10)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping])

Wynik uległ nieznacznej poprawie. Następnie zdefiniujemy funkcję, która pozwoli nam stworzyć przewidywania dla danych na podstawie modelu. 

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
lin_forecast = model_forecast(model, values[split - window_size:-1], window_size)[:, 0]
lin_forecast.shape

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time, values, label="Values")
plot_series(x_valid, lin_forecast, label="Forecast")

In [ ]:
print_error(y_valid, lin_forecast)

Rezultat z dość znaczną poprawą. Spróbujmy jednak poprawić naszą sieć, poprzez dodanie kolejnych wartsw, wykorzystamy w tym celu warstwy Dense oraz dodamy do nich funkcję aktywacji relu. Na początku spróbujemy sprawdzić jaka stała ucząca najlepiej sprawdzi się dla naszych danych. 

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(10, activation="relu", input_shape=[window_size]),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(1)
])

lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-7 * 10**(epoch / 20))
optimizer = keras.optimizers.SGD(lr=1e-7, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
history = model.fit(train_set, epochs=120, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-7, 1e-2, 0, 30])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size)
valid_set = window_dataset(y_valid, window_size)

model = keras.models.Sequential([
  keras.layers.Dense(10, activation="relu", input_shape=[window_size]),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(1)
])

optimizer = keras.optimizers.SGD(lr=1e-4, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
early_stopping = keras.callbacks.EarlyStopping(patience=15)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping])

In [ ]:
dense_forecast = model_forecast(
    model,
    values[split - window_size:-1],
    window_size)[:, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time, values, label="Values")
plot_series(x_valid, dense_forecast, label="Forecast")

In [ ]:
print_error(y_valid, dense_forecast)

Powinniśmy zaobserwować kolejną poprawę wyników. 


#RNN

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(y_train, window_size, batch_size=128)

model = keras.models.Sequential([
  keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
  keras.layers.SimpleRNN(100, return_sequences=True),
  keras.layers.SimpleRNN(100),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200.0)
])
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-7 * 10**(epoch / 20))
optimizer = keras.optimizers.SGD(lr=1e-7, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-7, 1e-4, 0, 30])

Zmienimy kształt okna zbioru danych, który zostanie użyty do uczenia sieci, tak aby każda z warstw RNN przekazywała do następnej warstwy wszystkie dane, co powinno przyspieszyc proces uczenia. 

Uwaga. Ten model może uczyć się dłużej - możesz spróbować zmniejszyć parametry, takie jak ilość epok, lub parametr callbacku Early Stopping, lecz może to skutkować mniejszą dokładnością modelu.

In [ ]:
def seq2seq_window_dataset(series, window_size, batch_size=32,
                           shuffle_buffer=1000):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = seq2seq_window_dataset(y_train, window_size,
                                   batch_size=128)
valid_set = seq2seq_window_dataset(y_valid, window_size,
                                   batch_size=128)

model = keras.models.Sequential([
  keras.layers.SimpleRNN(100, return_sequences=True,
                         input_shape=[None, 1]),
  keras.layers.SimpleRNN(100, return_sequences=True),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200.0)
])
optimizer = keras.optimizers.SGD(lr=1e-6, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
early_stopping = keras.callbacks.EarlyStopping(patience=15)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint", save_best_only=True)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping, model_checkpoint])

In [ ]:
rnn_forecast = model_forecast(model,values[..., np.newaxis], window_size)[split - window_size:-1,-1,0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time, values, label="Values")
plot_series(x_valid, rnn_forecast, label="Forecast")

In [ ]:
print_error(y_valid, rnn_forecast)

#LSTRM RNN

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = sequential_window_dataset(y_train, window_size)

model = keras.models.Sequential([
  keras.layers.LSTM(100, return_sequences=True, stateful=True,
                    batch_input_shape=[1, None, 1]),
  keras.layers.LSTM(100, return_sequences=True, stateful=True),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200.0)
])
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
reset_states = ResetStatesCallback()
optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
history = model.fit(train_set, epochs=100,
                    callbacks=[lr_schedule, reset_states])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 30])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = sequential_window_dataset(y_train, window_size)
valid_set = sequential_window_dataset(y_valid, window_size)

model = keras.models.Sequential([
  keras.layers.LSTM(100, return_sequences=True, stateful=True,
                         batch_input_shape=[1, None, 1]),
  keras.layers.LSTM(100, return_sequences=True, stateful=True),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200.0)
])
optimizer = keras.optimizers.SGD(lr=5e-7, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
reset_states = ResetStatesCallback()
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=15)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping, model_checkpoint, reset_states])

In [ ]:
model = keras.models.load_model("my_checkpoint.h5")

In [ ]:
rnn_forecast = model.predict(values[np.newaxis, :, np.newaxis])
rnn_forecast = rnn_forecast[0, split - 1:-1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time, values)
plot_series(x_valid, rnn_forecast)

In [ ]:
print_error(y_valid, rnn_forecast)

#CNN

Ostatnia sieć dokłada do warstw LSTM, warstwę konwolucyjną. Na początku  zdefiniujemy odpowiednią stałą uczącą w sposób analogiczny jak dotychczas. 

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = seq2seq_window_dataset(y_train, window_size,
                                   batch_size=128)

model = keras.models.Sequential([
  keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  keras.layers.LSTM(32, return_sequences=True),
  keras.layers.LSTM(32, return_sequences=True),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200)
])
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-4, 0, 30])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = seq2seq_window_dataset(y_train, window_size,
                                   batch_size=128)
valid_set = seq2seq_window_dataset(y_valid, window_size,
                                   batch_size=128)

model = keras.models.Sequential([
  keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  keras.layers.LSTM(32, return_sequences=True),
  keras.layers.LSTM(32, return_sequences=True),
  keras.layers.Dense(1),
  keras.layers.Lambda(lambda x: x * 200)
])
optimizer = keras.optimizers.SGD(lr=1e-5, momentum=0.9)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae", "mse"])

model_checkpoint = keras.callbacks.ModelCheckpoint(
    "my_checkpoint_cnn.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=15)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping, model_checkpoint])

In [ ]:
model = keras.models.load_model("my_checkpoint_cnn.h5")

In [ ]:
cnn_forecast = model_forecast(model, values[..., np.newaxis], window_size)
cnn_forecast = cnn_forecast[split - window_size:-1, -1, 0]

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time, values)
plot_series(x_valid, cnn_forecast)

In [ ]:
print_error(y_valid, cnn_forecast)

Osiągane wyniki są najlepsze spośród wszystkich modeli opartych na sieciach neuronowych, jednak metoda ARIMA osiąga ogółem najlepszy wynik dla użytych danych.